In [1]:
import tensorflow as tf
# #指定使用那块GUP训练
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
config = tf.ConfigProto()
# 设置最大占有GPU不超过显存的70%
config.gpu_options.per_process_gpu_memory_fraction = 0.7 
# # 重点：设置动态分配GPU
config.gpu_options.allow_growth = True
# 创建session时
tf.Session(config=config) 

In [2]:
import numpy as np
from keras.datasets import cifar10
from torch.utils.data import Dataset
import torch.utils.data as Data
class MyDataset(Dataset):
    def __init__(self, imgs, transform=None):
        # super().__init__()
        self.imgs = imgs
        self.transform = transform

    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, index):
        img = self.imgs[index]
        if self.transform is not None:
            img = self.transform(img)
        else:
            img = torch.from_numpy(img)
        img=img.reshape([3,32,32])
        return img
import torch
from torch.autograd import Variable
import model
import torch.nn.functional as F
model = model.cifar10(128)
model.load_state_dict(torch.load('./log/default/best-85.pth'))
model.cuda()
def EuclideanDistances(A, B):
    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(), ED.shape[0] * ED.shape[1])


def cal_distance_image_real(images, labels):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]
    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)


def cal_distance_image_fake(images):
    x_dataset = MyDataset(images)
    # print(x_dataset[0].shape)
    x_real_loader = Data.DataLoader(dataset=x_dataset, batch_size=200, shuffle=True)
    y_logits = []
    labels=[]
    for i, data in enumerate(x_real_loader):
        # indx_target = target.clone()
        data = data.cuda()
        data = Variable(data, volatile=True)
        output = model(data)
        pred = output.data.max(1)[1]
        labels += [i for i in pred.cpu().numpy()]
        pred = F.softmax(output).cpu().detach().numpy()
        y_logits += [i for i in pred]

    dict = {}
    all_dis = []
    for i in range(10):
        dict[i] = []
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i] = np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

import os
if not os.path.isdir('saved_models_{}'.format('bgan')):
    os.mkdir('saved_models_{}'.format('bgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('bgan'), mode='w')
import torch.utils.data as Data
import cv2

from keras.datasets import cifar10
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import keras.backend as K
from PIL import Image
import matplotlib.pyplot as plt
from keras.utils.generic_utils import Progbar
import sys

import numpy as np
import os
class BGAN():
    """Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/"""
    def __init__(self):
        self.img_rows = 32
        self.img_cols = 32
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss=self.boundary_loss, optimizer=optimizer)
    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 8 * 8, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((8, 8, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        # model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def boundary_loss(self, y_true, y_pred):
        """
        Boundary seeking loss.
        Reference: https://wiseodd.github.io/techblog/2017/03/07/boundary-seeking-gan/
        """
        return 0.5 * K.mean((K.log(y_pred) - K.log(1 - y_pred))**2)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        # X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step=0
        steps = []
        values = []

        for epoch in range(epochs):
            # ---------------------
            #  Train Discriminator
            # ---------------------
            for index in range(nb_batches):
                global_step+=1
                # progress_bar.update(index)

                # get a batch of real images
                image_batch = X_train[index * batch_size:(index + 1) * batch_size]
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(image_batch, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d global_step: %d[D loss: %f, acc.: %.2f%%] [G loss: %f]"
                      % (epoch,global_step, d_loss[0], 100 * d_loss[1], g_loss))
                sample_num=5000

                if global_step % sample_interval == 0:
                    self.mode_drop(X_test,y_test,sample_num,global_step)


    def mode_drop(self,x_test,y_test,sample_num, global_step):
        r, c = 10, 1000
        noise = np.random.normal(0, 1, (r * c, 100))
        # sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('step:' + str(global_step))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    bgan = BGAN()
    bgan.train(epochs=50, batch_size=64, sample_interval=200)


Using TensorFlow backend.
Sequential(
  (0): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (2): ReLU()
  (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (5): ReLU()
  (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (9): ReLU()
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  (12): ReLU()
  (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 8, 8, 64)          256       
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 global_step: 1[D loss: 1.072787, acc.: 29.69%] [G loss: 0.616631]
epoch:0 global_step: 2[D loss: 0.812416, acc.: 48.44%] [G loss: 0.779934]
epoch:0 global_step: 3[D loss: 0.765170, acc.: 52.34%] [G loss: 0.869669]
epoch:0 global_step: 4[D loss: 0.631528, acc.: 65.62%] [G loss: 0.901426]
epoch:0 global_step: 5[D loss: 0.605477, acc.: 67.97%] [G loss: 1.141541]
epoch:0 global_step: 6[D loss: 0.634294, acc.: 60.94%] [G loss: 0.775865]
epoch:0 global_step: 7[D loss: 0.646682, acc.: 64.84%] [G loss: 1.030896]
epoch:0 global_step: 8[D loss: 0.647635, acc.: 66.41%] [G loss: 1.288874]
epoch:0 global_step: 9[D loss: 0.712468, acc.: 58.59%] [G loss: 1.099649]
epoch:0 global_step: 10[D loss: 0.763221, acc.: 51.56%] [G loss: 1.018638]
epoch:0 global_step: 11[D loss: 0.828150, acc.: 51.56%] [G loss: 1.019334]
epoch:0 global_step: 12[D loss: 0.757943, acc.: 53.91%] [G loss: 0.874831]
epoch:0 global_step: 13[D loss: 0.809597, acc.: 47.66%] [G loss: 0.937510]
epoch:0 global_step: 14[D loss: 0.

epoch:0 global_step: 112[D loss: 0.758041, acc.: 58.59%] [G loss: 1.579435]
epoch:0 global_step: 113[D loss: 0.819409, acc.: 50.78%] [G loss: 1.223165]
epoch:0 global_step: 114[D loss: 0.590345, acc.: 67.97%] [G loss: 1.458520]
epoch:0 global_step: 115[D loss: 0.786538, acc.: 58.59%] [G loss: 1.275707]
epoch:0 global_step: 116[D loss: 0.734642, acc.: 59.38%] [G loss: 0.980329]
epoch:0 global_step: 117[D loss: 0.732803, acc.: 56.25%] [G loss: 1.010858]
epoch:0 global_step: 118[D loss: 0.781350, acc.: 54.69%] [G loss: 1.273874]
epoch:0 global_step: 119[D loss: 0.673166, acc.: 64.84%] [G loss: 1.470813]
epoch:0 global_step: 120[D loss: 0.638388, acc.: 67.97%] [G loss: 1.552302]
epoch:0 global_step: 121[D loss: 0.778735, acc.: 57.81%] [G loss: 1.823439]
epoch:0 global_step: 122[D loss: 0.596177, acc.: 68.75%] [G loss: 1.581921]
epoch:0 global_step: 123[D loss: 0.603589, acc.: 68.75%] [G loss: 1.335723]
epoch:0 global_step: 124[D loss: 0.670353, acc.: 64.06%] [G loss: 1.381492]
epoch:0 glob

...

epoch:49 global_step: 38530[D loss: 0.650444, acc.: 60.94%] [G loss: 0.679243]
epoch:49 global_step: 38531[D loss: 0.513018, acc.: 77.34%] [G loss: 0.451301]
epoch:49 global_step: 38532[D loss: 0.474440, acc.: 82.03%] [G loss: 0.641426]
epoch:49 global_step: 38533[D loss: 0.674156, acc.: 61.72%] [G loss: 0.917904]
epoch:49 global_step: 38534[D loss: 0.549605, acc.: 72.66%] [G loss: 0.541119]
epoch:49 global_step: 38535[D loss: 0.731140, acc.: 52.34%] [G loss: 0.521537]
epoch:49 global_step: 38536[D loss: 0.945610, acc.: 33.59%] [G loss: 0.495609]
epoch:49 global_step: 38537[D loss: 0.563767, acc.: 71.09%] [G loss: 0.504421]
epoch:49 global_step: 38538[D loss: 0.632414, acc.: 64.06%] [G loss: 0.567507]
epoch:49 global_step: 38539[D loss: 0.730511, acc.: 53.12%] [G loss: 0.320622]
epoch:49 global_step: 38540[D loss: 0.667759, acc.: 59.38%] [G loss: 0.600767]
epoch:49 global_step: 38541[D loss: 0.574233, acc.: 70.31%] [G loss: 0.741217]
epoch:49 global_step: 38542[D loss: 0.759249, acc.: 

epoch:49 global_step: 38634[D loss: 0.564482, acc.: 78.12%] [G loss: 0.624247]
epoch:49 global_step: 38635[D loss: 0.521430, acc.: 72.66%] [G loss: 0.439011]
epoch:49 global_step: 38636[D loss: 0.778987, acc.: 48.44%] [G loss: 0.342835]
epoch:49 global_step: 38637[D loss: 0.702716, acc.: 54.69%] [G loss: 1.082690]
epoch:49 global_step: 38638[D loss: 0.678775, acc.: 55.47%] [G loss: 0.527433]
epoch:49 global_step: 38639[D loss: 0.690362, acc.: 63.28%] [G loss: 0.329314]
epoch:49 global_step: 38640[D loss: 0.676021, acc.: 58.59%] [G loss: 0.439984]
epoch:49 global_step: 38641[D loss: 0.820574, acc.: 45.31%] [G loss: 0.365371]
epoch:49 global_step: 38642[D loss: 0.617810, acc.: 60.94%] [G loss: 0.507868]
epoch:49 global_step: 38643[D loss: 0.625901, acc.: 69.53%] [G loss: 0.466882]
epoch:49 global_step: 38644[D loss: 0.591256, acc.: 68.75%] [G loss: 0.698946]
epoch:49 global_step: 38645[D loss: 0.554485, acc.: 69.53%] [G loss: 0.465630]
epoch:49 global_step: 38646[D loss: 0.783987, acc.: 

epoch:49 global_step: 38741[D loss: 0.530279, acc.: 75.78%] [G loss: 0.620758]
epoch:49 global_step: 38742[D loss: 0.718401, acc.: 50.78%] [G loss: 0.590557]
epoch:49 global_step: 38743[D loss: 0.645199, acc.: 60.16%] [G loss: 0.581750]
epoch:49 global_step: 38744[D loss: 0.574232, acc.: 70.31%] [G loss: 0.536038]
epoch:49 global_step: 38745[D loss: 0.579071, acc.: 71.09%] [G loss: 0.477855]
epoch:49 global_step: 38746[D loss: 0.645744, acc.: 60.94%] [G loss: 0.333205]
epoch:49 global_step: 38747[D loss: 0.423217, acc.: 79.69%] [G loss: 0.478572]
epoch:49 global_step: 38748[D loss: 0.552112, acc.: 70.31%] [G loss: 0.508478]
epoch:49 global_step: 38749[D loss: 0.542000, acc.: 74.22%] [G loss: 0.317906]
epoch:49 global_step: 38750[D loss: 0.573657, acc.: 69.53%] [G loss: 0.861355]
epoch:49 global_step: 38751[D loss: 0.776673, acc.: 48.44%] [G loss: 0.658972]
epoch:49 global_step: 38752[D loss: 0.515565, acc.: 74.22%] [G loss: 0.909280]
epoch:49 global_step: 38753[D loss: 0.536363, acc.: 

epoch:49 global_step: 38845[D loss: 0.444143, acc.: 78.91%] [G loss: 0.489433]
epoch:49 global_step: 38846[D loss: 0.719619, acc.: 61.72%] [G loss: 0.654154]
epoch:49 global_step: 38847[D loss: 0.556849, acc.: 71.88%] [G loss: 0.632106]
epoch:49 global_step: 38848[D loss: 0.625439, acc.: 64.84%] [G loss: 0.650118]
epoch:49 global_step: 38849[D loss: 0.699603, acc.: 59.38%] [G loss: 0.341834]
epoch:49 global_step: 38850[D loss: 0.544335, acc.: 72.66%] [G loss: 0.320624]
epoch:49 global_step: 38851[D loss: 0.609412, acc.: 66.41%] [G loss: 0.387339]
epoch:49 global_step: 38852[D loss: 0.569970, acc.: 68.75%] [G loss: 0.279463]
epoch:49 global_step: 38853[D loss: 0.606331, acc.: 66.41%] [G loss: 0.702440]
epoch:49 global_step: 38854[D loss: 0.555212, acc.: 75.78%] [G loss: 0.465011]
epoch:49 global_step: 38855[D loss: 0.697405, acc.: 56.25%] [G loss: 1.104414]
epoch:49 global_step: 38856[D loss: 0.574975, acc.: 70.31%] [G loss: 0.400963]
epoch:49 global_step: 38857[D loss: 0.743325, acc.: 

epoch:49 global_step: 38952[D loss: 0.630934, acc.: 67.19%] [G loss: 0.303994]
epoch:49 global_step: 38953[D loss: 0.565922, acc.: 75.78%] [G loss: 0.522167]
epoch:49 global_step: 38954[D loss: 0.635721, acc.: 62.50%] [G loss: 0.405992]
epoch:49 global_step: 38955[D loss: 0.650963, acc.: 65.62%] [G loss: 0.508147]
epoch:49 global_step: 38956[D loss: 0.463287, acc.: 78.91%] [G loss: 0.645301]
epoch:49 global_step: 38957[D loss: 0.514776, acc.: 78.91%] [G loss: 0.851332]
epoch:49 global_step: 38958[D loss: 0.597483, acc.: 71.09%] [G loss: 0.707999]
epoch:49 global_step: 38959[D loss: 0.381548, acc.: 85.94%] [G loss: 0.505379]
epoch:49 global_step: 38960[D loss: 0.680164, acc.: 62.50%] [G loss: 0.715391]
epoch:49 global_step: 38961[D loss: 1.075162, acc.: 23.44%] [G loss: 0.314634]
epoch:49 global_step: 38962[D loss: 0.545872, acc.: 75.78%] [G loss: 0.765810]
epoch:49 global_step: 38963[D loss: 0.722880, acc.: 56.25%] [G loss: 0.925782]
epoch:49 global_step: 38964[D loss: 0.518349, acc.: 